# ONNX Model Converter

Notebook chuyển đổi model HuggingFace sang định dạng ONNX với quantization.

**Tính năng:**
- Hỗ trợ nhiều loại model: embedding (feature-extraction), reranker-llm (CausalLM), text-classification
- Quantization FP16 và INT8 Dynamic (tùy chọn)
- Tự động tạo Model Card
- Hướng dẫn upload lên HuggingFace Hub

**Cách sử dụng:**
1. Thay đổi cấu hình ở Section 2
2. Chạy tất cả các cell từ đầu đến cuối
3. Upload model theo hướng dẫn ở Section 9

**Lưu ý:**
- Mỗi lần chạy chỉ xử lý 1 model
- Qwen3-Reranker sử dụng architecture CausalLM (không phải classification head)
- Nên chạy trên Kaggle hoặc máy có ít nhất 16GB RAM cho models lớn

## 1. Bootstrap - Environment Setup

Cell này tự động phát hiện và cấu hình môi trường (local/colab/kaggle)

In [ ]:
!pip uninstall -y onnxruntime onnxruntime-gpu

In [ ]:
# === BOOTSTRAP CELL - UNIVERSAL SETUP ===
import sys
import os
from pathlib import Path

# GitHub configuration
GITHUB_USER = "n24q02m"
REPO_NAME = "n24q02m-kaggle-competitions"
BRANCH = "main"


# Detect environment
def detect_env():
    if "google.colab" in sys.modules:
        return "colab"
    elif "kaggle_web_client" in sys.modules or os.path.exists("/kaggle"):
        return "kaggle"
    else:
        return "local"


ENV = detect_env()
print(f"Detected: {ENV.upper()}")

# Setup theo môi trường
if ENV == "local":
    # Local: Import trực tiếp từ repo
    repo_root = Path.cwd().parent
    if str(repo_root) not in sys.path:
        sys.path.insert(0, str(repo_root))

    from core import setup_env

    env = setup_env.setup()

else:
    # Cloud: Download setup_env.py từ GitHub
    import requests

    CORE_URL = (
        f"https://raw.githubusercontent.com/{GITHUB_USER}/{REPO_NAME}/{BRANCH}/core"
    )

    # Download setup_env.py
    print("Downloading setup_env.py...")
    response = requests.get(f"{CORE_URL}/setup_env.py")
    with open("setup_env.py", "w") as f:
        f.write(response.text)

    # Import và setup
    import setup_env

    env = setup_env.setup(GITHUB_USER, REPO_NAME)

# Hiển thị thông tin môi trường
env.info()

## 2. Cấu Hình Model

Thay đổi các giá trị bên dưới để cấu hình model cần chuyển đổi.

In [ ]:
# ============================================================
# CẤU HÌNH - THAY ĐỔI CÁC GIÁ TRỊ NÀY
# ============================================================

# Model HuggingFace cần chuyển đổi
MODEL_ID = "Qwen/Qwen3-Reranker-0.6B"

# Task của model (auto = tự động detect, hoặc chọn thủ công)
# Các giá trị:
#   - "auto": Tự động detect từ config
#   - "feature-extraction": Embedding models (output: hidden states)
#   - "reranker-llm": Reranker models dựa trên CausalLM (Qwen3-Reranker, output: logits -> yes/no scoring)
#   - "text-classification": Classification models với head riêng
# Lưu ý: Qwen3-Reranker cần dùng "reranker-llm" (dựa trên LLM, không phải classification head)
TASK = "auto"

# Tên output (để trống = tự động tạo từ MODEL_ID)
OUTPUT_NAME = ""

# HuggingFace username của bạn
HF_USERNAME = "n24q02m"

# Bật/tắt FP16 export (giảm kích thước ~50%, cần GPU để inference nhanh)
ENABLE_FP16 = False

# Bật/tắt INT8 quantization (giảm kích thước ~75%, inference trên CPU nhanh)
ENABLE_INT8 = True

# Legacy compatibility (deprecated, sẽ xóa sau)
ENABLE_QUANTIZATION = ENABLE_INT8

# ============================================================
# KHÔNG CẦN THAY ĐỔI PHẦN DƯỚI
# ============================================================

## 3. Import Thư Viện

Sử dụng PyTorch và ONNX Runtime trực tiếp (không dùng optimum)

In [ ]:
import gc
import shutil
import warnings
from pathlib import Path

import numpy as np
import torch
import onnx
import onnxruntime as ort
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModel,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
)

warnings.filterwarnings("ignore")


def clear_memory():
    """Giải phóng bộ nhớ RAM và GPU"""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()


print(f"PyTorch: {torch.__version__}")
print(f"ONNX: {onnx.__version__}")
print(f"ONNX Runtime: {ort.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 4. Xử Lý Cấu Hình

In [ ]:
# Tự động detect task từ model_id
def detect_task(model_id):
    """Tự động detect task dựa trên tên model"""
    model_lower = model_id.lower()

    # Qwen Reranker sử dụng LLM architecture (yes/no generation)
    if "qwen" in model_lower and "reranker" in model_lower:
        return "reranker-llm"
    elif "embedding" in model_lower:
        return "feature-extraction"
    elif "reranker" in model_lower or "ranker" in model_lower:
        # Reranker khác có thể dùng classification head
        return "text-classification"
    elif "classifier" in model_lower or "classification" in model_lower:
        return "text-classification"
    else:
        # Thử đọc config để detect
        try:
            config = AutoConfig.from_pretrained(model_id, trust_remote_code=True)
            if hasattr(config, "num_labels") and config.num_labels > 0:
                return "text-classification"
        except:
            pass
        return "feature-extraction"


# Xử lý cấu hình
if TASK == "auto":
    TASK = detect_task(MODEL_ID)
    print(f"Task tự động detect: {TASK}")

if not OUTPUT_NAME:
    # Tạo tên từ MODEL_ID
    OUTPUT_NAME = MODEL_ID.split("/")[-1] + "-ONNX"

# Thiết lập thư mục output theo môi trường
if ENV == "colab":
    OUTPUT_DIR = Path("/content/models")
elif ENV == "kaggle":
    OUTPUT_DIR = Path("/kaggle/working/models")
else:
    OUTPUT_DIR = Path.cwd() / "models"

# Tạo thư mục cho model
MODEL_OUTPUT_DIR = OUTPUT_DIR / OUTPUT_NAME.lower().replace(" ", "-")
FP32_DIR = MODEL_OUTPUT_DIR / "fp32"
FP16_DIR = MODEL_OUTPUT_DIR / "fp16"
INT8_DIR = MODEL_OUTPUT_DIR / "int8"

MODEL_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
FP32_DIR.mkdir(parents=True, exist_ok=True)
if ENABLE_FP16:
    FP16_DIR.mkdir(parents=True, exist_ok=True)
if ENABLE_INT8:
    INT8_DIR.mkdir(parents=True, exist_ok=True)

# Chọn model class và output names phù hợp
if TASK == "feature-extraction":
    ModelClass = AutoModel
    OUTPUT_NAMES = ["last_hidden_state"]
elif TASK == "reranker-llm":
    # Qwen3-Reranker sử dụng LLM (CausalLM) với yes/no scoring
    ModelClass = AutoModelForCausalLM
    OUTPUT_NAMES = ["logits"]
elif TASK == "text-classification":
    ModelClass = AutoModelForSequenceClassification
    OUTPUT_NAMES = ["logits"]
else:
    raise ValueError(f"Task không được hỗ trợ: {TASK}")

print("\n" + "=" * 50)
print("CẤU HÌNH CUỐI CÙNG")
print("=" * 50)
print(f"Model ID: {MODEL_ID}")
print(f"Task: {TASK}")
print(f"Model Class: {ModelClass.__name__}")
print(f"Output Name: {OUTPUT_NAME}")
print(f"Output Dir: {MODEL_OUTPUT_DIR}")
print(f"FP32: True")
print(f"FP16: {ENABLE_FP16}")
print(f"INT8: {ENABLE_INT8}")
print(f"HF Username: {HF_USERNAME}")

## 5. Export Model sang ONNX (FP32)

Sử dụng `torch.onnx.export` trực tiếp thay vì optimum

In [ ]:
print("=" * 50)
print("EXPORT ONNX (FP32)")
print("=" * 50)

print(f"\nĐang tải và export model: {MODEL_ID}")
print(f"Task: {TASK}")
print("Sử dụng torch.onnx.export trực tiếp")

export_success = False
ONNX_FILE = FP32_DIR / "model.onnx"

try:
    # Tải tokenizer
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
    tokenizer.save_pretrained(FP32_DIR)

    # Tải model PyTorch
    print("\nĐang tải model PyTorch...")
    model = ModelClass.from_pretrained(MODEL_ID, trust_remote_code=True)
    model.eval()

    # Tạo dummy input dựa trên task
    if TASK == "reranker-llm":
        # Qwen3-Reranker cần format đặc biệt với chat template
        dummy_query = "What is AI?"
        dummy_doc = "AI is artificial intelligence."
        dummy_text = f"<Instruct>: Given a query, judge if the document is relevant.\n<Query>: {dummy_query}\n<Document>: {dummy_doc}"
        dummy_inputs = tokenizer(
            dummy_text, return_tensors="pt", padding="max_length", max_length=128
        )
    else:
        dummy_text = "This is a test sentence for ONNX export."
        dummy_inputs = tokenizer(
            dummy_text, return_tensors="pt", padding="max_length", max_length=128
        )

    # Chuẩn bị input cho export
    input_names = ["input_ids", "attention_mask"]
    dynamic_axes = {
        "input_ids": {0: "batch_size", 1: "sequence_length"},
        "attention_mask": {0: "batch_size", 1: "sequence_length"},
    }

    # Thêm position_ids nếu model cần (Qwen models)
    model_config = model.config
    if (
        hasattr(model_config, "model_type")
        and "qwen" in model_config.model_type.lower()
    ):
        dummy_inputs["position_ids"] = torch.arange(
            dummy_inputs["input_ids"].shape[1]
        ).unsqueeze(0)
        input_names.append("position_ids")
        dynamic_axes["position_ids"] = {0: "batch_size", 1: "sequence_length"}

    # Dynamic axes cho output
    for out_name in OUTPUT_NAMES:
        if out_name == "last_hidden_state":
            dynamic_axes[out_name] = {0: "batch_size", 1: "sequence_length"}
        elif out_name == "logits":
            if TASK == "reranker-llm":
                # CausalLM: [batch, seq_len, vocab_size]
                dynamic_axes[out_name] = {0: "batch_size", 1: "sequence_length"}
            else:
                # Classification: [batch, num_labels]
                dynamic_axes[out_name] = {0: "batch_size"}

    # Export sang ONNX
    print("Đang export sang ONNX...")
    with torch.no_grad():
        torch.onnx.export(
            model,
            tuple(dummy_inputs.values()),
            str(ONNX_FILE),
            input_names=input_names,
            output_names=OUTPUT_NAMES,
            dynamic_axes=dynamic_axes,
            opset_version=14,
            do_constant_folding=True,
        )

    # Lưu config
    model.config.save_pretrained(FP32_DIR)

    # Kiểm tra file ONNX
    if ONNX_FILE.exists():
        # Verify ONNX model
        onnx_model = onnx.load(str(ONNX_FILE))
        onnx.checker.check_model(onnx_model)
        del onnx_model

        print(f"\nModel đã được export tại: {FP32_DIR}")
        export_success = True

        # Hiển thị files
        print("\nDanh sách files:")
        total_size = 0
        for f in FP32_DIR.iterdir():
            if f.is_file():
                size_mb = f.stat().st_size / (1024**2)
                total_size += size_mb
                print(f"  - {f.name} ({size_mb:.2f} MB)")
        print(f"  Tổng: {total_size:.2f} MB")

    # Giải phóng bộ nhớ
    del model, tokenizer
    clear_memory()

except Exception as e:
    print(f"\nLỗi khi export: {e}")
    import traceback

    traceback.print_exc()
    clear_memory()

if export_success:
    print("\nExport FP32: THÀNH CÔNG")
else:
    print("\nExport FP32: THẤT BẠI")
    raise Exception("Không thể export model sang ONNX")

## 6. Quantization (FP16 và INT8)

Áp dụng FP16 conversion và/hoặc INT8 Dynamic Quantization để giảm kích thước model.

In [ ]:
quantize_success = False


def get_onnx_size(folder):
    """Tính tổng kích thước ONNX (bao gồm cả external data)"""
    total = 0
    for f in Path(folder).iterdir():
        if f.is_file() and (f.suffix == ".onnx" or f.name.endswith("_data")):
            total += f.stat().st_size
    return total / (1024**2)


# --- FP16 Quantization ---
if ENABLE_FP16:
    print("=" * 50)
    print("QUANTIZATION (FP16)")
    print("=" * 50)

    print("\nĐang convert model sang FP16...")

    try:
        from onnxconverter_common import float16

        # Load FP32 model
        onnx_fp32 = onnx.load(str(ONNX_FILE))

        # Convert to FP16
        onnx_fp16 = float16.convert_float_to_float16(
            onnx_fp32,
            keep_io_types=True,  # Giữ input/output FP32 để compatibility
            min_positive_val=1e-7,
            max_finite_val=1e4,
        )

        # Save FP16 model
        fp16_model_path = FP16_DIR / "model.onnx"
        onnx.save(onnx_fp16, str(fp16_model_path))

        del onnx_fp32, onnx_fp16

        # Copy tokenizer và config từ FP32
        for f in FP32_DIR.iterdir():
            if (
                f.is_file()
                and not f.name.endswith(".onnx")
                and not f.name.endswith("_data")
            ):
                shutil.copy(f, FP16_DIR / f.name)

        if fp16_model_path.exists():
            print("\nFP16 Conversion: THÀNH CÔNG")

            print("\nDanh sách files FP16:")
            total_size = 0
            for f in FP16_DIR.iterdir():
                if f.is_file():
                    size_mb = f.stat().st_size / (1024**2)
                    total_size += size_mb
                    print(f"  - {f.name} ({size_mb:.2f} MB)")
            print(f"  Tổng: {total_size:.2f} MB")

    except Exception as e:
        print(f"\nLỗi FP16 conversion: {e}")
        import traceback

        traceback.print_exc()
    finally:
        clear_memory()


# --- INT8 Quantization ---
if ENABLE_INT8:
    print("=" * 50)
    print("QUANTIZATION (INT8)")
    print("=" * 50)

    print("\nĐang quantize model sang INT8...")
    print("Sử dụng onnxruntime.quantization trực tiếp")

    from onnxruntime.quantization import quantize_dynamic, QuantType

    try:
        output_model = INT8_DIR / "model.onnx"

        print(f"  Input: {ONNX_FILE}")
        print(f"  Output: {output_model}")

        # Quantize dynamic với INT8
        quantize_dynamic(
            model_input=str(ONNX_FILE),
            model_output=str(output_model),
            weight_type=QuantType.QInt8,
            per_channel=False,
            reduce_range=False,
            extra_options={
                "MatMulConstBOnly": True,
            },
        )

        # Kiểm tra kết quả
        if output_model.exists():
            print("\nINT8 Quantization: THÀNH CÔNG")
            quantize_success = True

            # Copy tokenizer files
            for f in FP32_DIR.iterdir():
                if not f.name.endswith(".onnx") and not f.name.endswith("_data"):
                    if f.is_file() and not (INT8_DIR / f.name).exists():
                        shutil.copy(f, INT8_DIR / f.name)

            # Hiển thị files
            print("\nDanh sách files INT8:")
            total_size = 0
            for f in INT8_DIR.iterdir():
                if f.is_file():
                    size_mb = f.stat().st_size / (1024**2)
                    total_size += size_mb
                    print(f"  - {f.name} ({size_mb:.2f} MB)")
            print(f"  Tổng: {total_size:.2f} MB")

        else:
            print("\nINT8 Quantization: THẤT BẠI - File output không tồn tại")

    except Exception as e:
        print(f"\nLỗi khi quantize INT8: {e}")
        import traceback

        traceback.print_exc()

    finally:
        clear_memory()


# --- Summary ---
print("\n" + "=" * 50)
print("TỔNG KẾT QUANTIZATION")
print("=" * 50)

fp32_size = get_onnx_size(FP32_DIR)
print(f"\nFP32: {fp32_size:.2f} MB")

if ENABLE_FP16 and (FP16_DIR / "model.onnx").exists():
    fp16_size = get_onnx_size(FP16_DIR)
    reduction_fp16 = (1 - fp16_size / fp32_size) * 100 if fp32_size > 0 else 0
    print(f"FP16: {fp16_size:.2f} MB (giảm {reduction_fp16:.1f}%)")

if ENABLE_INT8 and quantize_success:
    int8_size = get_onnx_size(INT8_DIR)
    reduction_int8 = (1 - int8_size / fp32_size) * 100 if fp32_size > 0 else 0
    print(f"INT8: {int8_size:.2f} MB (giảm {reduction_int8:.1f}%)")

## 7. Kiểm Tra Model

Sử dụng ONNX Runtime trực tiếp (không dùng optimum)

In [ ]:
print("=" * 50)
print("KIỂM TRA MODEL")
print("=" * 50)


def test_embedding_model(model_path, model_name):
    """Test embedding model ONNX"""
    try:
        onnx_file = model_path / "model.onnx"
        if not onnx_file.exists():
            print(f"{model_name}: Không tìm thấy file ONNX")
            return False

        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained(model_path)

        # Tạo ONNX Runtime session
        session = ort.InferenceSession(
            str(onnx_file), providers=["CPUExecutionProvider"]
        )

        # Lấy input/output names
        input_names = [inp.name for inp in session.get_inputs()]

        # Test text
        test_text = "This is a test sentence for embedding."
        inputs = tokenizer(
            test_text, return_tensors="np", padding=True, truncation=True
        )

        # Chuẩn bị feed dict
        feed_dict = {}
        for name in input_names:
            if name == "input_ids":
                feed_dict[name] = inputs["input_ids"].astype(np.int64)
            elif name == "attention_mask":
                feed_dict[name] = inputs["attention_mask"].astype(np.int64)
            elif name == "position_ids":
                seq_len = inputs["input_ids"].shape[1]
                feed_dict[name] = np.arange(seq_len, dtype=np.int64).reshape(1, -1)

        # Run inference
        outputs = session.run(None, feed_dict)
        last_hidden_state = outputs[0]

        # Mean pooling để lấy embedding
        attention_mask = inputs["attention_mask"]
        mask_expanded = np.expand_dims(attention_mask, -1).astype(np.float32)
        sum_embeddings = np.sum(last_hidden_state * mask_expanded, axis=1)
        sum_mask = np.clip(np.sum(mask_expanded, axis=1), a_min=1e-9, a_max=None)
        embeddings = sum_embeddings / sum_mask

        print(f"{model_name}: OK")
        print(f"  - Hidden state shape: {last_hidden_state.shape}")
        print(f"  - Embedding shape: {embeddings.shape}")
        print(f"  - Embedding norm: {np.linalg.norm(embeddings):.4f}")

        del session
        return True

    except Exception as e:
        print(f"{model_name}: LỖI - {e}")
        import traceback

        traceback.print_exc()
        return False


def test_reranker_model(model_path, model_name, tokenizer_orig):
    """Test reranker model ONNX (CausalLM style với yes/no scoring)"""
    try:
        onnx_file = model_path / "model.onnx"
        if not onnx_file.exists():
            print(f"{model_name}: Không tìm thấy file ONNX")
            return False

        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained(model_path)

        # Lấy token IDs cho yes/no
        token_true_id = tokenizer.convert_tokens_to_ids("yes")
        token_false_id = tokenizer.convert_tokens_to_ids("no")

        # Tạo ONNX Runtime session
        session = ort.InferenceSession(
            str(onnx_file), providers=["CPUExecutionProvider"]
        )

        input_names = [inp.name for inp in session.get_inputs()]

        # Test query và document
        test_query = "What is machine learning?"
        test_doc = "Machine learning is a branch of artificial intelligence that enables computers to learn from data."

        # Format theo Qwen3-Reranker
        prefix = "<Instruct>: Given a query and a document, judge whether the document is relevant to the query. Output only 'yes' or 'no'."
        test_text = f"{prefix}\n<Query>: {test_query}\n<Document>: {test_doc}"

        inputs = tokenizer(
            test_text, return_tensors="np", padding=True, truncation=True
        )

        # Chuẩn bị feed dict
        feed_dict = {}
        for name in input_names:
            if name == "input_ids":
                feed_dict[name] = inputs["input_ids"].astype(np.int64)
            elif name == "attention_mask":
                feed_dict[name] = inputs["attention_mask"].astype(np.int64)
            elif name == "position_ids":
                seq_len = inputs["input_ids"].shape[1]
                feed_dict[name] = np.arange(seq_len, dtype=np.int64).reshape(1, -1)

        # Run inference
        outputs = session.run(None, feed_dict)
        logits = outputs[0]  # [batch, seq_len, vocab_size]

        # Lấy logits của token cuối cùng
        last_logits = logits[0, -1, :]  # [vocab_size]

        # Tính score từ yes/no logits
        true_logit = last_logits[token_true_id]
        false_logit = last_logits[token_false_id]

        # Softmax để có probability
        max_logit = max(true_logit, false_logit)
        exp_true = np.exp(true_logit - max_logit)
        exp_false = np.exp(false_logit - max_logit)
        score = exp_true / (exp_true + exp_false)

        print(f"{model_name}: OK")
        print(f"  - Logits shape: {logits.shape}")
        print(f"  - Yes logit: {true_logit:.4f}, No logit: {false_logit:.4f}")
        print(f"  - Relevance score: {score:.4f}")

        del session
        return True

    except Exception as e:
        print(f"{model_name}: LỖI - {e}")
        import traceback

        traceback.print_exc()
        return False


# Test các model
if TASK == "feature-extraction":
    print("\n--- Test FP32 ---")
    fp32_ok = test_embedding_model(FP32_DIR, "FP32")

    if ENABLE_FP16 and (FP16_DIR / "model.onnx").exists():
        print("\n--- Test FP16 ---")
        fp16_ok = test_embedding_model(FP16_DIR, "FP16")
    else:
        fp16_ok = False

    if ENABLE_INT8 and quantize_success:
        print("\n--- Test INT8 ---")
        int8_ok = test_embedding_model(INT8_DIR, "INT8")
    else:
        int8_ok = False

elif TASK == "reranker-llm":
    # Load original tokenizer for token IDs
    tokenizer_orig = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)

    print("\n--- Test FP32 ---")
    fp32_ok = test_reranker_model(FP32_DIR, "FP32", tokenizer_orig)

    if ENABLE_FP16 and (FP16_DIR / "model.onnx").exists():
        print("\n--- Test FP16 ---")
        fp16_ok = test_reranker_model(FP16_DIR, "FP16", tokenizer_orig)
    else:
        fp16_ok = False

    if ENABLE_INT8 and quantize_success:
        print("\n--- Test INT8 ---")
        int8_ok = test_reranker_model(INT8_DIR, "INT8", tokenizer_orig)
    else:
        int8_ok = False

else:
    print(f"Chưa hỗ trợ test cho task: {TASK}")
    fp32_ok = fp16_ok = int8_ok = False

clear_memory()

## 8. Tạo Model Card

Cập nhật README với hướng dẫn sử dụng ONNX Runtime trực tiếp

In [ ]:
print("=" * 50)
print("TẠO MODEL CARD")
print("=" * 50)

# Xác định model type và pipeline tag
if TASK == "feature-extraction":
    model_type = "embedding"
    pipeline_tag = "feature-extraction"
elif TASK == "reranker-llm":
    model_type = "reranker"
    pipeline_tag = "text-classification"  # Không có pipeline_tag riêng cho reranker
else:
    model_type = "model"
    pipeline_tag = TASK

# Xác định versions
versions = ["FP32"]
if ENABLE_FP16 and (FP16_DIR / "model.onnx").exists():
    versions.append("FP16")
if ENABLE_INT8 and quantize_success:
    versions.append("INT8")

# Code example cho từng task
if TASK == "feature-extraction":
    usage_code = """```python
import numpy as np
import onnxruntime as ort
from transformers import AutoTokenizer

# Load tokenizer va model
tokenizer = AutoTokenizer.from_pretrained("{hf_user}/{output_name}", subfolder="fp32")
session = ort.InferenceSession("{hf_user}/{output_name}/fp32/model.onnx")

# Inference
text = "This is a test sentence."
inputs = tokenizer(text, return_tensors="np", padding=True, truncation=True)

# Chuan bi input (them position_ids cho Qwen models)
feed_dict = {{
    "input_ids": inputs["input_ids"].astype(np.int64),
    "attention_mask": inputs["attention_mask"].astype(np.int64),
    "position_ids": np.arange(inputs["input_ids"].shape[1], dtype=np.int64).reshape(1, -1)
}}

outputs = session.run(None, feed_dict)
last_hidden_state = outputs[0]  # Shape: (batch_size, seq_len, hidden_size)

# Mean pooling de lay embedding
attention_mask = inputs["attention_mask"]
mask_expanded = np.expand_dims(attention_mask, -1).astype(np.float32)
sum_embeddings = np.sum(last_hidden_state * mask_expanded, axis=1)
sum_mask = np.clip(np.sum(mask_expanded, axis=1), a_min=1e-9, a_max=None)
embeddings = sum_embeddings / sum_mask  # Shape: (batch_size, hidden_size)
```""".format(hf_user=HF_USERNAME, output_name=OUTPUT_NAME)

elif TASK == "reranker-llm":
    usage_code = """```python
import numpy as np
import onnxruntime as ort
from transformers import AutoTokenizer

# Load tokenizer va model
tokenizer = AutoTokenizer.from_pretrained("{hf_user}/{output_name}", subfolder="fp32")
session = ort.InferenceSession("{hf_user}/{output_name}/fp32/model.onnx")

# Lay token IDs cho yes/no
token_true_id = tokenizer.convert_tokens_to_ids("yes")
token_false_id = tokenizer.convert_tokens_to_ids("no")

# Chuan bi input
query = "What is machine learning?"
document = "Machine learning is a branch of AI that enables computers to learn from data."

prefix = "<Instruct>: Given a query and a document, judge whether the document is relevant to the query. Output only 'yes' or 'no'."
text = f"{{prefix}}\\n<Query>: {{query}}\\n<Document>: {{document}}"

inputs = tokenizer(text, return_tensors="np", padding=True, truncation=True)

feed_dict = {{
    "input_ids": inputs["input_ids"].astype(np.int64),
    "attention_mask": inputs["attention_mask"].astype(np.int64),
    "position_ids": np.arange(inputs["input_ids"].shape[1], dtype=np.int64).reshape(1, -1)
}}

# Run inference
outputs = session.run(None, feed_dict)
logits = outputs[0]  # Shape: (batch_size, seq_len, vocab_size)

# Lay logits cua token cuoi cung
last_logits = logits[0, -1, :]

# Tinh relevance score tu yes/no logits
true_logit = last_logits[token_true_id]
false_logit = last_logits[token_false_id]

# Softmax
max_logit = max(true_logit, false_logit)
exp_true = np.exp(true_logit - max_logit)
exp_false = np.exp(false_logit - max_logit)
relevance_score = exp_true / (exp_true + exp_false)

print(f"Relevance score: {{relevance_score:.4f}}")
```""".format(hf_user=HF_USERNAME, output_name=OUTPUT_NAME)

else:
    usage_code = "See original model documentation."

# Tạo available versions section
versions_section = "- `fp32/`: Full precision (FP32)\n"
if ENABLE_FP16 and (FP16_DIR / "model.onnx").exists():
    versions_section += "- `fp16/`: Half precision (FP16)\n"
if ENABLE_INT8 and quantize_success:
    versions_section += "- `int8/`: Quantized INT8 (dynamic quantization)\n"

model_card = f"""---
license: apache-2.0
tags:
  - onnx
  - {model_type}
base_model: {MODEL_ID}
pipeline_tag: {pipeline_tag}
---

# {OUTPUT_NAME}

ONNX version of [{MODEL_ID}](https://huggingface.co/{MODEL_ID})

Converted using PyTorch `torch.onnx.export` and quantized with `onnxruntime.quantization`.

## Model Information

| Property | Value |
|----------|-------|
| Source Model | [{MODEL_ID}](https://huggingface.co/{MODEL_ID}) |
| Format | ONNX |
| Versions | {" + ".join(versions)} |
| Task | {TASK} |

## Available Versions

{versions_section}

## Usage with ONNX Runtime

{usage_code}

## License

Apache 2.0 (following the original model license)

## Credits

- Original model: [{MODEL_ID}](https://huggingface.co/{MODEL_ID})
- ONNX conversion: PyTorch + ONNX Runtime
"""

# Lưu Model Card
readme_path = MODEL_OUTPUT_DIR / "README.md"
readme_path.write_text(model_card, encoding="utf-8")
print(f"Đã tạo: {readme_path}")

## 9. Hướng Dẫn Upload

In [ ]:
print("=" * 50)
print("HƯỚNG DẪN UPLOAD")
print("=" * 50)

print(f"""
=== CÁCH 1: HuggingFace CLI ===

# Đăng nhập
huggingface-cli login

# Tạo repo
huggingface-cli repo create {OUTPUT_NAME} --type model

# Upload
huggingface-cli upload {HF_USERNAME}/{OUTPUT_NAME} "{MODEL_OUTPUT_DIR}" .

=== CÁCH 2: Web UI ===

1. Truy cập: https://huggingface.co/new
2. Tạo repo: {OUTPUT_NAME}
3. Upload files từ: {MODEL_OUTPUT_DIR}

=== CẤU TRÚC FILES ===
""")

# Hiển thị cấu trúc
for item in sorted(MODEL_OUTPUT_DIR.rglob("*")):
    rel_path = item.relative_to(MODEL_OUTPUT_DIR)
    indent = "  " * (len(rel_path.parts) - 1)
    if item.is_file():
        size_mb = item.stat().st_size / (1024**2)
        print(f"{indent}{item.name} ({size_mb:.2f} MB)")
    else:
        print(f"{indent}{item.name}/")

## 10. Tổng Kết

In [ ]:
print("\n" + "=" * 50)
print("TỔNG KẾT")
print("=" * 50)

print(f"\nModel: {MODEL_ID}")
print(f"Task: {TASK}")
print(f"Output: {MODEL_OUTPUT_DIR}")


# Kích thước (bao gồm external data)
def get_folder_size(folder):
    return sum(f.stat().st_size for f in Path(folder).rglob("*") if f.is_file()) / (
        1024**2
    )


print(f"\nKích thước:")
try:
    fp32_total = get_folder_size(FP32_DIR)
    print(f"  FP32: {fp32_total:.2f} MB")

    if ENABLE_FP16 and FP16_DIR.exists() and (FP16_DIR / "model.onnx").exists():
        fp16_total = get_folder_size(FP16_DIR)
        print(f"  FP16: {fp16_total:.2f} MB")
        if fp32_total > 0:
            reduction = (1 - fp16_total / fp32_total) * 100
            print(f"        Giảm: {reduction:.1f}%")

    if ENABLE_INT8 and INT8_DIR.exists() and (INT8_DIR / "model.onnx").exists():
        int8_total = get_folder_size(INT8_DIR)
        print(f"  INT8: {int8_total:.2f} MB")
        if fp32_total > 0:
            reduction = (1 - int8_total / fp32_total) * 100
            print(f"        Giảm: {reduction:.1f}%")

    total_size = get_folder_size(MODEL_OUTPUT_DIR)
    print(f"  Tổng: {total_size:.2f} MB")
except Exception as e:
    print(f"  Lỗi tính kích thước: {e}")

print(f"\nKết quả test:")
print(f"  FP32: {'OK' if fp32_ok else 'LỖI'}")
if ENABLE_FP16:
    print(f"  FP16: {'OK' if fp16_ok else 'KHÔNG KHẢ DỤNG'}")
if ENABLE_INT8:
    print(f"  INT8: {'OK' if quantize_success and int8_ok else 'KHÔNG KHẢ DỤNG'}")

print(f"\nURL sau khi upload:")
print(f"  https://huggingface.co/{HF_USERNAME}/{OUTPUT_NAME}")

print("\n" + "=" * 50)
print("HOÀN THÀNH!")
print("=" * 50)